<a href="https://colab.research.google.com/github/Herbsta/Blackjack/blob/main/Blackjack_Heuristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#Use for card shuffle
import random
 
#Hearts, Diamonds, Clubs, Spade
suits = ('H', 'D', 'C', 'S')
 
#Possible card ranks
#ranking = ('A', '2', '3', '4','5', '6', '7', '8', '9', '10', 'J', 'Q','K')
ranking = ('2','2','2','A','A','A','A','A','A')
 
#Point values dict 
card_val = { 'A': 11, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10 }
 
class PlayerBank:
  def __init__(self,chip_pool,id):
    self.chip_pool = chip_pool
    self.bet = 0
    self.win = 0
    self.id = id
 
#Card Class
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return self.suit + self.rank
    
    def grab_suit(self):
        return self.suit
    
    def grab_rank(self):
        return self.rank
    
    def draw(self):
        print(self.suit + self.rank)
        
        
#Hand Class       
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.isFinished = False
        self.surrendered = False
        self.isBust = False
        self.blackjack = False
        self.isDoubledDown = False
        
        
    def __str__(self): 
        '''Returns a string of current hand composition'''
        hand_comp = "" 
        
        for card in self.cards:
            card_name = card.__str__()
            hand_comp += " " + card_name 
            
        return 'The hand has %s' %hand_comp
    
    def card_add(self, card,aceAlways11 = False): 
     '''Adds another card to the hand'''
     ace = 0
     self.cards.append(card)
     #calculate the new value
 
     self.value = 0
     for card in self.cards:
       v = card_val[card.rank]
       if 'A' == card.rank:
         ace += 1
            
       self.value += v
 
        
     while True:
       if self.value > 21:
         if ace > 0:
           self.value -= 10
           ace -=1
         else:
           print("Bust")
           print(self.__str__())
           self.isBust = True
           self.isFinished = True
       else:
         break

        
    def draw(self, hidden):
        if hidden == True:
          #Don't show first hidden card
          starting_card = 1
        else: 
          starting_card = 0
          for x in range(starting_card, len(self.cards)):
            self.cards[x].draw()
                
#Deck Class
class Deck: 
    def __init__(self,numOfDecks):
        '''Creates a deck in order'''
        self.deck = []
        for suit in suits: 
            for rank in ranking: 
                self.deck.append(Card(suit, rank))
        self.deck = self.deck * numOfDecks
        
                
    def shuffle(self): 
        '''Shuffles the deck, without using Python's built-in method'''
        random.shuffle(self.deck)
        
    def deal(self):
        '''Grabs the first item from the deck'''
        single_card = self.deck.pop()
        
        return single_card
    
    def __str__(self):
        deck_comp = "" 
        for card in self.deck: 
            deck_comp += " " + card.__str__()
            
        return "The deck has" + deck_comp
    
#First Bet
def make_bet(player):
    print("Player ID: {}".format(player.id))
    '''Asks the player for the bet amount and '''
    player.bet = 0
    player.win = 0
    
    print(' What amount of chips would you like to bet? (Enter whole integer please) ')
    
    #While loop to keep asking for the bet
    while player.bet == 0:
        bet_comp = input() #Use bet_comp as a checker 
        bet_comp = float(bet_comp)
        #Check to make sure the bet is within the remaining amount of chips left
        if bet_comp >= 1 and bet_comp <= player.chip_pool:
            player.bet = bet_comp
        else: 
            print("Invalid bet, you only have " + str(player.chip_pool) + " remaining")
    return player
 
def Split(deck,handArray,id,index):
  hasSplit = False
  if handArray[id][index].cards[0].rank == handArray[id][index].cards[1].rank:
    option = input("Player {}, do you want to split? y/n ".format(id))
    if option.lower() == 'y':
      hasSplit = True
      # make 2 new hands and destroy the old one and add those card to the new hands
      cards = handArray[id][index]
      handArray[id].append(Hand())
      handArray[id][-1].card_add(cards.cards[0],aceAlways11 = True)
      handArray[id][-1].card_add(deck.deal(),aceAlways11 = True)
      handArray[id].append(Hand())
      handArray[id][-1].card_add(cards.cards[1],aceAlways11 = True)
      handArray[id][-1].card_add(deck.deal(),aceAlways11 = True)
      handArray[id].pop(index)
    
  return deck, handArray, hasSplit
 
            
def deal_cards(players,deckCount,maxSplitDecks):
    '''This function deals out cards and sets up round''' 
    #set up dictionaries for player's hand(s)
    handDict = {}
    for i in players:
      handDict[i.id] = []
    
 
    #Create a deck
    deck = Deck(deckCount)
    #Shuffle it
    deck.shuffle()
    #Set up bet
    for x,i in enumerate(players):
      players[x] = make_bet(i)
            
    #Set up both player and dealer hands 
 
    for i in players:
      handDict[i.id].append(Hand())
 
    dealer_hand = Hand()
    
    #Deal out initial cards 
    for i in players:
      handDict[i.id][0].card_add(deck.deal(),aceAlways11 = True)
      handDict[i.id][0].card_add(deck.deal(),aceAlways11 = True)
      
      #print draw
      print("Player {}'s Hand: ".format(i.id))
      handDict[i.id][0].draw(hidden=False)
 
    
    dealer_hand.card_add(deck.deal(), aceAlways11 = True)
    dealer_hand.card_add(deck.deal(), aceAlways11 = True)
    print("Dealer's Hand: ")
    print(dealer_hand.cards[0].__str__())
 
    #check for natural
    for i in players:
      if handDict[i.id][0].value == 21:
        handDict[i.id][0].isFinished = True
        handDict[i.id][0].blackjack = True
    
    if dealer_hand.value == 21:
      dealer_hand.blackjack = True
      dealer_hand.isFinished = True
 
    #check for early surrender
    for i in players:
      if handDict[i.id][0].isFinished:
        continue
      option = input("Player {}, do you want to surrender? y/n".format(i.id))
      if option.lower() == 'y':
        handDict[i.id][0].isFinished = True
        handDict[i.id][0].surrendered = True
 
 
    for i in players:
      # Check for split
      #skips natural blacks
      if handDict[i.id][0].isFinished:
        continue
 
      splitCount = 1
      j = 0
      while j < len(handDict[i.id]):
        #can't split if they dont have enough money
        if i.bet * 2 > i.chip_pool:
          break
        deck,handDict, hasSplit = Split(deck,handDict,i.id,j)
        if hasSplit:
          print("Split Hand is: ",end="")
          print(handDict[i.id][j].__str__())
          splitCount += 1
          if splitCount == maxSplitDecks: break
        else:
          j += 1
 
 
    # Check for double
    
    for i in players:
      for x,j in enumerate(handDict[i.id]):
        #skips natural blacks
        if j.isFinished:
          continue
        canDouble = False
        if i.bet * 2 > i.chip_pool:
          continue

        if doubleTotals == "All":
          canDouble = True
        else:

          for k in doubleTotals:
            if j.value == k:
              canDouble = True
              break
        
        if canDouble:
          option = input("Player {}, do you want to double down? y/n".format(i.id))
          if option.lower() == "y":
            handDict[i.id][x].card_add(deck.deal())
            handDict[i.id][x].isFinished = True
            handDict[i.id][x].isDoubledDown = True
 
    #Hit or Stay Check
 
    for i in players:
      for x,j in enumerate(handDict[i.id]):
        while handDict[i.id][x].isFinished == False:
          
          print("Player {}'s hand: ".format(i.id),end="")
          print(j.__str__())
          option = input("Player {}, do you want to stay or hit? s/h".format(i.id))
          if option.lower() == 's':
            handDict[i.id][x].isFinished = True
          else:
            handDict[i.id][x].card_add(deck.deal(),aceAlways11=False)
 
    print(" ")
    #Dealer's Go
 
    while dealer_hand.isFinished == False:
      #check for soft 17
      if dealer_hand.value < 17:
        dealer_hand.card_add(deck.deal(),aceAlways11=True)
      else:
        dealer_hand.isFinished = True
    
    print("Dealer's Hand: ")
    print(dealer_hand.__str__())
 
    #Compare hands and winnings
    for x,i in enumerate(players):
      for y,j in enumerate(handDict[i.id]):

        #did double down?
        if j.isDoubledDown:
          dd = 2
        else:
          dd = 1
        
        #check for surrender
        if j.surrendered:
          players[x].win -= players[x].bet /2
          continue
          
        #check if busted
        if j.isBust:
          '''no winnings'''
          players[x].win -= players[x].bet * dd
          
        elif dealer_hand.blackjack:
          #check for dealer blackjack
          if j.blackjack:
            '''push'''
            
          else:
            '''dealer won'''
            players[x].win -= players[x].bet * dd
            
        elif j.blackjack:
          '''player blackjack'''
          players[x].win += players[x].bet * 1.5
        
        elif dealer_hand.isBust:
          ''' Dealer busted, so player wins'''
          players[x].win += players[x].bet * dd
          
        elif dealer_hand.value < j.value:
          '''player has higher value than dealer'''
          players[x].win += players[x].bet * dd
        elif j.value < dealer_hand.value:
          '''dealer has higher value than player'''
          players[x].win -= players[x].bet * dd
 
        elif j.value == dealer_hand.value:
          '''push'''
      
      players[x].chip_pool += players[x].win   
            
    for i in players:
      for j in handDict[i.id]:
        print("Player {}".format(i.id))
        print(j.__str__())
        print(j.value)
    return players
 
def PrintWinnings(players):
  for i in players:
    print("Player {} collection: {}".format(i.id,i.win))
    print("Player {} new chip pool: {}".format(i.id,i.chip_pool))

In [46]:
'''The following code will initiate the game.''' 
global doubleTotals
doubleTotals = "All" #for double down on any 2 1st cards

#doubleTotals = [10,11] 
DeckNumber = 6
reSplitNumber = 2
startChips = 100

players = []
for i in range(2): 
  players.append(PlayerBank(startChips,i)) 
for i in range(5): 
  players = deal_cards(players,DeckNumber,reSplitNumber) 
  PrintWinnings(players)

  for i in players:
    if i.chip_pool <= 0.0:
      players.remove(i)

Player ID: 0
 What amount of chips would you like to bet? (Enter whole integer please) 
6
Player ID: 1
 What amount of chips would you like to bet? (Enter whole integer please) 
6
Player 0's Hand: 
H2
D2
Player 1's Hand: 
SA
DA
Dealer's Hand: 
SA
Player 0, do you want to surrender? y/nN
Player 1, do you want to surrender? y/nN
Player 0, do you want to split? y/n N
Player 1, do you want to split? y/n N
Player 0, do you want to double down? y/nN
Player 1, do you want to double down? y/nN
Player 0's hand: The hand has  H2 D2
Player 0, do you want to stay or hit? s/hH
Player 0's hand: The hand has  H2 D2 H2
Player 0, do you want to stay or hit? s/hH
Player 0's hand: The hand has  H2 D2 H2 DA
Player 0, do you want to stay or hit? s/hH
Player 0's hand: The hand has  H2 D2 H2 DA DA
Player 0, do you want to stay or hit? s/hH
Player 0's hand: The hand has  H2 D2 H2 DA DA HA
Player 0, do you want to stay or hit? s/hH
Player 0's hand: The hand has  H2 D2 H2 DA DA HA C2
Player 0, do you want to st

KeyboardInterrupt: ignored